<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [1]:
 !pip install -U torchtext==0.8.1

     |████████████████████████████████| 7.0MB 23.4MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchtext 0.9.0
    Uninstalling torchtext-0.9.0:
      Successfully uninstalled torchtext-0.9.0


In [2]:
import pandas as pd
import numpy as np
import torch

from torchtext import datasets

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [108]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [4]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [109]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [110]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [8]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.rnn = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        packed_output, hidden = self.rnn(packed_embedded)
        #unpack sequence
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  
        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim = 1)
        hidden = self.dropout(hidden)  
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
        return self.fc(hidden)

Поиграйтесь с гиперпараметрами

In [9]:
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3

In [10]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [11]:
model = model.to(device)

In [12]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

max_epochs = 20

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [13]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        opt.zero_grad()
        input_embed = batch.text[0].to(device)
        text_length = batch.text[1].cpu()
        preds = model(input_embed,text_length).requires_grad_()
        loss = loss_func(preds,batch.label.reshape(-1,1))
        loss.backward()
        opt.step()
        train_loss += loss.item()
    train_loss /= len(train_iter)

    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        # YOUR CODE GOES HERE
        with torch.no_grad():
          input_embed = batch.text[0].to(device)
          text_length = batch.text[1].cpu()
          preds = model(input_embed,text_length)
          loss = loss_func(preds,batch.label.reshape(-1,1))
          val_loss += loss.item()
    val_loss /= len(valid_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1, Training Loss: 0.6263296522580795, Validation Loss: 0.5473223523063174


Epoch: 2, Training Loss: 0.37254699776424977, Validation Loss: 0.4177126239164401


Epoch: 3, Training Loss: 0.213188671595315, Validation Loss: 0.39303600459785787


Epoch: 4, Training Loss: 0.09701077092139826, Validation Loss: 0.524324987146814


Epoch: 5, Training Loss: 0.03197074199798959, Validation Loss: 0.6397510105270451


<All keys matched successfully>

Посчитайте f1-score вашего классификатора на тестовом датасете.

Ответ: **0.8006687275071515**

In [14]:
from sklearn.metrics import f1_score

test_acc = 0.0
model.train()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
for it, batch in pbar:
  with torch.no_grad():
    input_embed = batch.text[0].to(device)
    text_length = batch.text[1].cpu()
    preds = model(input_embed,text_length)
    preds = np.where(preds.cpu() > 0, 1, 0)
    loss = f1_score(batch.label.reshape(-1,1).cpu(),preds)
    test_acc += loss.item()
test_acc /= len(test_iter)
print('ACC',test_acc)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


ACC 0.8006687275071515


In [17]:
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [18]:
from sklearn.metrics import f1_score

all_preds = []
all_labels = []
test_acc = 0.0
model.train()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
for it, batch in pbar:
  with torch.no_grad():
    input_embed = batch.text[0].to(device)
    text_length = batch.text[1].cpu()
    preds = model(input_embed,text_length)
    preds = np.array([sigmoid(float(element[0])) for element in preds]).reshape(-1,1)
    preds = np.where(preds > 0.5, 1, 0)
    all_preds.append(preds)
    all_labels.append(batch.label.reshape(-1,1))
test_acc = f1_score(np.vstack([element.cpu() for element in all_labels]),np.vstack(all_preds))
# test_acc += loss.item()
#test_acc /= len(test_iter)
print('ACC',test_acc)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


ACC 0.8415004248775947


## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [19]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes fo

In [20]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [21]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)

        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(kernel_sizes[0],emb_dim))
        
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(kernel_sizes[1],emb_dim))
        
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=(kernel_sizes[2],emb_dim))
        
        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        
        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self, text):

        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)  # may be reshape here
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))  # may be reshape here
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))  # may be reshape here
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
            
        return self.fc(cat)

In [22]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels = 64
dropout = 0.5
dim = 300

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [23]:
model.to(device)

CNN(
  (embedding): Embedding(201966, 300)
  (conv_0): Conv2d(1, 64, kernel_size=(3, 300), stride=(1, 1))
  (conv_1): Conv2d(1, 64, kernel_size=(4, 300), stride=(1, 1))
  (conv_2): Conv2d(1, 64, kernel_size=(5, 300), stride=(1, 1))
  (fc): Linear(in_features=192, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [24]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

In [25]:
max_epochs = 30

Обучите!

In [26]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        opt.zero_grad()
        input_embed = batch.text.to(device)
        preds = model(input_embed).requires_grad_()
        loss = loss_func(preds,batch.label.reshape(-1,1))
        loss.backward()
        opt.step()
        train_loss += loss.item()
    train_loss /= len(train_iter)

    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        with torch.no_grad():
          input_embed = batch.text.to(device)
          preds = model(input_embed)
          loss = loss_func(preds,batch.label.reshape(-1,1))
          val_loss += loss.item()
    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1, Training Loss: 0.6447382980454577, Validation Loss: 0.47999321619669594


Epoch: 2, Training Loss: 0.49907878759133556, Validation Loss: 0.4316095620393753


Epoch: 3, Training Loss: 0.42415861187190035, Validation Loss: 0.3914056827624639


Epoch: 4, Training Loss: 0.3700983419905614, Validation Loss: 0.37390600542227426


Epoch: 5, Training Loss: 0.31463586348686773, Validation Loss: 0.3634883066018422


Epoch: 6, Training Loss: 0.2469343848689629, Validation Loss: 0.33489260971546175


Epoch: 7, Training Loss: 0.1898144791378592, Validation Loss: 0.34097914695739745


Epoch: 8, Training Loss: 0.13864192350284896, Validation Loss: 0.3482154736916224


<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

Ответ: **0.81352359331928**

In [27]:
from sklearn.metrics import f1_score

all_preds = []
all_labels = []
test_acc = 0.0
model.train()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
for it, batch in pbar:
  with torch.no_grad():
    input_embed = batch.text.to(device)
    preds = model(input_embed)
    preds = np.array([sigmoid(float(element[0])) for element in preds]).reshape(-1,1)
    preds = np.where(preds > 0.5, 1, 0)
    all_preds.append(preds)
    all_labels.append(batch.label.reshape(-1,1))
test_acc = f1_score(np.vstack([element.cpu() for element in all_labels]),np.vstack(all_preds))
print('ACC',test_acc)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


ACC 0.81352359331928


## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [28]:
!pip install -q captum

     |████████████████████████████████| 4.4MB 19.4MB/s 


In [45]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [46]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [47]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)
interpret_sentence(model, 'I love this film thanks wooooow!!!!', label=1)

pred:  pos ( 0.99 ) , delta:  tensor([6.3440e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.45 ) , delta:  tensor([8.3728e-06], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.98 ) , delta:  tensor([3.5981e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.44 ) , delta:  tensor([8.2025e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.67 ) , delta:  tensor([1.8926e-05], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [48]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.99),pos,1.26,It was a fantastic performance ! pad
pos,neg (0.45),pos,-0.59,Best film ever pad pad pad pad
pos,pos (0.98),pos,1.39,Such a great show! pad pad pad
neg,neg (0.01),pos,-0.99,It was a horrible movie pad pad
neg,neg (0.44),pos,-0.33,I've never watched something as bad pad
neg,neg (0.01),pos,-0.91,It is a disgusting movie! pad pad
pos,pos (0.67),pos,1.56,I love this film thanks wooooow!!!! pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.99),pos,1.26,It was a fantastic performance ! pad
pos,neg (0.45),pos,-0.59,Best film ever pad pad pad pad
pos,pos (0.98),pos,1.39,Such a great show! pad pad pad
neg,neg (0.01),pos,-0.99,It was a horrible movie pad pad
neg,neg (0.44),pos,-0.33,I've never watched something as bad pad
neg,neg (0.01),pos,-0.91,It is a disgusting movie! pad pad
pos,pos (0.67),pos,1.56,I love this film thanks wooooow!!!! pad


## Эмбэдинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [52]:
from torchtext.vocab import GloVe
vec = GloVe(name='6B', dim=300)

In [35]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

device = "cuda" if torch.cuda.is_available() else "cpu"

train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [121]:
TEXT.build_vocab(train, vectors=vec)# YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(train)
word_embeddings = TEXT.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300

In [ ]:
#import gensim.downloader as api
#word2vec = api.load("glove-twitter-50")

In [ ]:
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout)

word_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

with torch.no_grad():
  for idx, word in enumerate(TEXT.vocab.itos):
        if word in vec.itos:
            model.embedding.weight[idx] = vec.get_vecs_by_tokens(tokens=[word]).to(device)

assert prev_shape == model.embedding.weight.shape
model.to(device)

opt = torch.optim.Adam(model.parameters())

Вы знаете, что делать.

In [133]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
        opt.zero_grad()
        input_embed = batch.text.to(device)
        preds = model(input_embed).requires_grad_()
        loss = loss_func(preds,batch.label.reshape(-1,1))
        loss.backward()
        opt.step()
        train_loss += loss.item()
    train_loss /= len(train_iter)

    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
        with torch.no_grad():
          input_embed = batch.text.to(device)
          preds = model(input_embed)
          loss = loss_func(preds,batch.label.reshape(-1,1))
          val_loss += loss.item()
    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Epoch: 1, Training Loss: 0.6593926857857808, Validation Loss: 0.4808348258336385


Epoch: 2, Training Loss: 0.503206689644904, Validation Loss: 0.43146848877271016


Epoch: 3, Training Loss: 0.4357806372381475, Validation Loss: 0.39606156448523205


Epoch: 4, Training Loss: 0.37696607204249305, Validation Loss: 0.37670205533504486


...

Epoch: 6, Training Loss: 0.25692274078835536, Validation Loss: 0.3365169068177541


Epoch: 7, Training Loss: 0.20282455778470004, Validation Loss: 0.33450498382250465


Epoch: 8, Training Loss: 0.15183764565600097, Validation Loss: 0.33968308170636496


Epoch: 9, Training Loss: 0.1026461258476233, Validation Loss: 0.3510115444660187


<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

Ответ: **0.8164241578632649**

In [134]:
from sklearn.metrics import f1_score

all_preds = []
all_labels = []
test_acc = 0.0
model.train()
pbar = tqdm(enumerate(test_iter), total=len(test_iter), leave=False)
for it, batch in pbar:
  with torch.no_grad():
    input_embed = batch.text.to(device)
    preds = model(input_embed)
    preds = np.array([sigmoid(float(element[0])) for element in preds]).reshape(-1,1)
    preds = np.where(preds > 0.5, 1, 0)
    all_preds.append(preds)
    all_labels.append(batch.label.reshape(-1,1))
test_acc = f1_score(np.vstack([element.cpu() for element in all_labels]),np.vstack(all_preds))
print('ACC',test_acc)

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


ACC 0.8164241578632649


Проверим насколько все хорошо!

In [148]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.96 ) , delta:  tensor([2.8041e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.98 ) , delta:  tensor([1.8883e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 1.00 ) , delta:  tensor([4.5667e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.02 ) , delta:  tensor([3.7771e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.24 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.75 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)


In [149]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,0.40,It was a fantastic performance ! pad
pos,pos (0.98),pos,1.03,Best film ever pad pad pad pad
pos,pos (1.00),pos,0.71,Such a great show! pad pad pad
neg,neg (0.02),pos,-0.49,It was a horrible movie pad pad
neg,neg (0.24),pos,-0.64,I've never watched something as bad pad
neg,pos (0.75),pos,-0.10,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,0.40,It was a fantastic performance ! pad
pos,pos (0.98),pos,1.03,Best film ever pad pad pad pad
pos,pos (1.00),pos,0.71,Such a great show! pad pad pad
neg,neg (0.02),pos,-0.49,It was a horrible movie pad pad
neg,neg (0.24),pos,-0.64,I've never watched something as bad pad
neg,pos (0.75),pos,-0.10,It is a disgusting movie! pad pad
